# We want to calculate the "similarity" of two records

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import isnull, when, count

In [2]:
dataset = spark.read.parquet("../preston-amazon/data-processed/core.parquet")
totalRows = dataset.count()
print("Number of rows: " + str(totalRows))

Number of rows: 3858


In [3]:
#test = dataset.filter(dataset["`http://rs.tdwg.org/dwc/terms/scientificName`"]=="Abascantus grandis Becker 1977")

In [4]:
#test.show(1, vertical=True)

In [5]:
df = dataset.toPandas()

In [6]:
# Get a subset of the data using occurence IDresults
#matchColumn = "http://rs.tdwg.org/dwc/terms/occurrenceID"
#matchValue = "244-37465-1175"

# Get a subset of the data using scientific name
matchColumn = "http://rs.tdwg.org/dwc/terms/scientificName"
matchValue = "Charax sp. “Madeira”"

subset = df[df[matchColumn] == matchValue]
subset = subset.dropna(axis=1)
subset = subset.loc[:, subset.columns.str.contains("dwc")]
subset

,http://rs.tdwg.org/dwc/terms/datasetID,http://rs.tdwg.org/dwc/terms/specificEpithet,http://rs.tdwg.org/dwc/terms/country,http://rs.tdwg.org/dwc/terms/basisOfRecord,http://rs.tdwg.org/dwc/terms/scientificName,http://rs.tdwg.org/dwc/terms/decimalLatitude,http://rs.tdwg.org/dwc/terms/eventDate,http://rs.tdwg.org/dwc/terms/waterBody,http://rs.tdwg.org/dwc/terms/locationID,http://rs.tdwg.org/dwc/terms/taxonRank,...,http://rs.tdwg.org/dwc/terms/year,http://rs.tdwg.org/dwc/terms/eventID,http://rs.tdwg.org/dwc/terms/decimalLongitude,http://rs.tdwg.org/dwc/terms/locality,http://rs.tdwg.org/dwc/terms/collectionCode,http://rs.tdwg.org/dwc/terms/family,http://rs.tdwg.org/dwc/terms/dynamicProperties,http://rs.tdwg.org/dwc/terms/eventRemarks,http://rs.tdwg.org/dwc/terms/occurrenceID,http://rs.tdwg.org/dwc/terms/genus
1,doi:10.13148/bfe105,sp. “Madeira”,Bolivia,PreservedSpecimen,Charax sp. “Madeira”,-12.578,28/07/02,Itenez-Blanco,BM3,species,...,2002,BM3-37465,-64.14555,Itenez-Blanco lake,CIRA,Characidae,waterBodyType=lake,Site was sampled on 2 consecutive days. Date r...,244-37465-1175,Charax
18,doi:10.13148/bfe105,sp. “Madeira”,Bolivia,PreservedSpecimen,Charax sp. “Madeira”,-13.01595,12/08/02,Itenez,IteR1,species,...,2002,IteR1-37480,-62.76304,Itenez river channel,CIRA,Characidae,waterBodyType=river channel,Site was sampled on 2 consecutive days. Date r...,643-37480-1175,Charax
249,doi:10.13148/bfe105,sp. “Madeira”,Bolivia,HumanObservation,Charax sp. “Madeira”,-12.51685,07/08/02,Itenez-Curichá,CurR,species,...,2002,CurR-37475,-63.92103,Itenez-Curichá river channel,CIRA,Characidae,waterBodyType=river channel,Site was sampled on 2 consecutive days. Date r...,454-37475,Charax
261,doi:10.13148/bfe105,sp. “Madeira”,Bolivia,PreservedSpecimen,Charax sp. “Madeira”,-12.48644,08/08/02,Itenez,IteM2,species,...,2002,IteM2-37476,-63.88531,Itenez lake,CIRA,Characidae,waterBodyType=lake,Site was sampled on 2 consecutive days. Date r...,585-37476-1175,Charax
430,doi:10.13148/bfe105,sp. “Madeira”,Bolivia,PreservedSpecimen,Charax sp. “Madeira”,-13.06051,25/07/02,Itenez-Blanco,BM2,species,...,2002,BM2-37462,-63.7789,Itenez-Blanco lake,CIRA,Characidae,waterBodyType=lake,Site was sampled on 2 consecutive days. Date r...,175-37462-1175,Charax
491,doi:10.13148/bfe105,sp. “Madeira”,Bolivia,PreservedSpecimen,Charax sp. “Madeira”,-14.89727,14/08/07,Mamoré-Ibare,RP3,species,...,2007,RP3-39308,-64.96466,Mamoré-Ibare river channel,CIRA,Characidae,waterBodyType=river channel,Site was sampled on 2 consecutive days. Date r...,1822-39308-1552,Charax
609,doi:10.13148/bfe105,sp. “Madeira”,Bolivia,PreservedSpecimen,Charax sp. “Madeira”,-13.17601,17/07/02,Itenez-San Antonio,IL1,species,...,2002,IL1-37454,-63.25462,Itenez-San Antonio lake,CIRA,Characidae,waterBodyType=lake,Site was sampled on 2 consecutive days. Date r...,740-37454-1175,Charax
665,doi:10.13148/bfe105,sp. “Madeira”,Bolivia,PreservedSpecimen,Charax sp. “Madeira”,-13.05622,11/08/02,Itenez,IteM1,species,...,2002,IteM1-37479,-62.75549,Itenez lake,CIRA,Characidae,waterBodyType=lake,Site was sampled on 2 consecutive days. Date r...,537-37479-1175,Charax
683,doi:10.13148/bfe105,sp. “Madeira”,Bolivia,HumanObservation,Charax sp. “Madeira”,-13.29413,21/07/02,Itenez-San Martín,SMR2,species,...,2002,SMR2-37458,-63.52765,Itenez-San Martín river channel,CIRA,Characidae,waterBodyType=river channel,Site was sampled on 2 consecutive days. Date r...,1063-37458,Charax
727,doi:10.13148/bfe105,sp. “Madeira”,Bolivia,HumanObservation,Charax sp. “Madeira”,-13.87487,25/08/07,Mamoré-Apere,RP4a,species,...,2007,RP4a-39319,-65.29235,Mamoré-Apere river channel,CIRA,Characidae,waterBodyType=river channel,Site was sampled on 2 consecutive days. Date r...,1896-39319,Charax


In [7]:
def getFieldSimilarities(a, b, function, args):
    df = pd.concat([a, b], axis=1, join="inner").transpose()
    return df.apply(lambda x: function(x.iloc[0], x.iloc[1], **args))

def getRecordSimilarity(a, b, comparisonFunction, cfArgs, aggregateFunction, afArgs):
    fieldSimilarities = getFieldSimilarities(a, b, comparisonFunction, cfArgs)
    recordSimilarity = aggregateFunction(fieldSimilarities, **afArgs)
    return [fieldSimilarities, recordSimilarity]

### Field value similarity functions
These functions compare two different string values for the same field. The prefix "fs_" is used for "field similarity".

In [8]:
def fs_sequenceMatcher(a, b, fieldName=""):
    from difflib import SequenceMatcher
    return SequenceMatcher(None, a, b).ratio()

### Record similarity functions
These functions calculate the overall similarity of two records by aggregating the computed similarity values for the fields they have in common. The prefix "rs_" is used for "record similarity".

In [9]:
def rs_average(fieldSimilarities):
    return np.average(fieldSimilarities)

def rs_thresholdAverage(fieldSimilarities, threshold=.9):
    return np.average(fieldSimilarities.apply(lambda x: x if x >= threshold else 0))

## Test similarity algorithms

In [10]:
recordA = subset.iloc[0]
recordB = subset.iloc[1]

print("----- Record A -----\n", recordA, "\n")
print("-results---- Record B -----\n", recordB)

----- Record A -----
 http://rs.tdwg.org/dwc/terms/datasetID                                          doi:10.13148/bfe105
http://rs.tdwg.org/dwc/terms/specificEpithet                                          sp. “Madeira”
http://rs.tdwg.org/dwc/terms/country                                                        Bolivia
http://rs.tdwg.org/dwc/terms/basisOfRecord                                        PreservedSpecimen
http://rs.tdwg.org/dwc/terms/scientificName                                    Charax sp. “Madeira”
http://rs.tdwg.org/dwc/terms/decimalLatitude                                                -12.578
http://rs.tdwg.org/dwc/terms/eventDate                                                     28/07/02
http://rs.tdwg.org/dwc/terms/waterBody                                                Itenez-Blanco
http://rs.tdwg.org/dwc/terms/locationID                                                         BM3
http://rs.tdwg.org/dwc/terms/taxonRank                                        

### Using SequenceMatching and a simple average

In [11]:
[fieldSimilarities, recordSimilarity] = getRecordSimilarity(
    recordA, recordB,
    comparisonFunction = fs_sequenceMatcher,
    cfArgs = {},
    aggregateFunction = rs_average,
    afArgs = {})

print("Field value similarities:\n", fieldSimilarities, "\n")
print("Overall similarity:\n", recordSimilarity)

Field value similarities:
 http://rs.tdwg.org/dwc/terms/datasetID            1.000000
http://rs.tdwg.org/dwc/terms/specificEpithet      1.000000
http://rs.tdwg.org/dwc/terms/country              1.000000
http://rs.tdwg.org/dwc/terms/basisOfRecord        1.000000
http://rs.tdwg.org/dwc/terms/scientificName       1.000000
http://rs.tdwg.org/dwc/terms/decimalLatitude      0.500000
http://rs.tdwg.org/dwc/terms/eventDate            0.625000
http://rs.tdwg.org/dwc/terms/waterBody            0.631579
http://rs.tdwg.org/dwc/terms/locationID           0.000000
http://rs.tdwg.org/dwc/terms/taxonRank            1.000000
http://rs.tdwg.org/dwc/terms/institutionCode      1.000000
http://rs.tdwg.org/dwc/terms/kingdom              1.000000
http://rs.tdwg.org/dwc/terms/year                 1.000000
http://rs.tdwg.org/dwc/terms/eventID              0.400000
http://rs.tdwg.org/dwc/terms/decimalLongitude     0.333333
http://rs.tdwg.org/dwc/terms/locality             0.473684
http://rs.tdwg.org/dwc/terms/

### Using SequenceMatching with a threshold and a simple average

In [12]:
# Threshold = .5
[fieldSimilarities, recordSimilarity] = getRecordSimilarity(
    recordA, recordB,
    comparisonFunction = fs_sequenceMatcher,
    cfArgs = {},
    aggregateFunction = rs_thresholdAverage,
    afArgs = {"threshold": .5})

print("Field value similarities:\n", fieldSimilarities, "\n")
print("Overall similarity:\n", recordSimilarity)

Field value similarities:
 http://rs.tdwg.org/dwc/terms/datasetID            1.000000
http://rs.tdwg.org/dwc/terms/specificEpithet      1.000000
http://rs.tdwg.org/dwc/terms/country              1.000000
http://rs.tdwg.org/dwc/terms/basisOfRecord        1.000000
http://rs.tdwg.org/dwc/terms/scientificName       1.000000
http://rs.tdwg.org/dwc/terms/decimalLatitude      0.500000
http://rs.tdwg.org/dwc/terms/eventDate            0.625000
http://rs.tdwg.org/dwc/terms/waterBody            0.631579
http://rs.tdwg.org/dwc/terms/locationID           0.000000
http://rs.tdwg.org/dwc/terms/taxonRank            1.000000
http://rs.tdwg.org/dwc/terms/institutionCode      1.000000
http://rs.tdwg.org/dwc/terms/kingdom              1.000000
http://rs.tdwg.org/dwc/terms/year                 1.000000
http://rs.tdwg.org/dwc/terms/eventID              0.400000
http://rs.tdwg.org/dwc/terms/decimalLongitude     0.333333
http://rs.tdwg.org/dwc/terms/locality             0.473684
http://rs.tdwg.org/dwc/terms/

In [13]:
# Threshold = .9
[fieldSimilarities, recordSimilarity] = getRecordSimilarity(
    recordA, recordB,
    comparisonFunction = fs_sequenceMatcher,
    cfArgs = {},
    aggregateFunction = rs_thresholdAverage,
    afArgs = {"threshold": .9})

#print("Field value similarities:\n", fieldSimilarities, "\n")
print("Overall similarity:\n", recordSimilarity)

Overall similarity:
 0.5909090909090909


In [14]:
# Threshold = 1.0
[fieldSimilarities, recordSimilarity] = getRecordSimilarity(
    recordA, recordB,
    comparisonFunction = fs_sequenceMatcher,
    cfArgs = {},
    aggregateFunction = rs_thresholdAverage,
    afArgs = {"threshold": 1})

#print("Field value similarities:\n", fieldSimilarities, "\n")
print("Overall similarity:\n", recordSimilarity)

Overall similarity:
 0.5909090909090909


## Need to compare results with a pair of duplicate records